# INFORMATION TO USER
This notebook generates the results referred to in **Table 2** of the manuscript in the **Results** subsection titled **Red Blood Cell Object Detection Model** to create the SSD300 Object Detection model. This notebook **MUST** be run on the **Google Cloud Computing Platform**. 

All versions are already built into the codes. 

# Training

## Setup

### Install

In [ ]:
# Compatibility with object detection API training script
!pip install tensorflow==1.15
!pip install tensorflow-gpu==1.15

# Fix for "InvalidArgumentError: TypeError: 'numpy.float64' object cannot be interpreted as an integer"
# Reference: https://stackoverflow.com/questions/59493606/pytorch-and-tensorflow-object-detection-evaluate-object-of-type-class-nump
!pip install numpy==1.17.4
!pip install pycocotools

Get `tensorflow/models` or `cd` to parent directory of the repository.

In [ ]:
import os
import pathlib


CWD = pathlib.Path.cwd()

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

print()
print(f'Current Working Directory: "{CWD}"')

Fixes for Training section

In [ ]:
# Fix for "ImportError: No module named nets"
# Reference: https://github.com/tensorflow/models/issues/1842
!cp -r {CWD}/models/research/slim/nets {CWD}/models/research/object_detection

In [ ]:
# Fix for "ValueError: anchor_strides must be a list with the same length as self._box_specs"
# Reference: https://stackoverflow.com/questions/53563976/tensorflow-object-detection-api-valueerror-anchor-strides-must-be-a-list-wit
# Modify "/.../models/research/object_detection/anchor_generators/multiple_grid_anchor_generator.py"
!cp {CWD}/multiple_grid_anchor_generator_fix.py {CWD}/models/research/object_detection/anchor_generators/multiple_grid_anchor_generator.py

Compile protobufs and install the object_detection package

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [ ]:
%%bash
cd models/research
pip install .

### Imports

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

Import the object detection module.

In [ ]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

Patches:

In [ ]:
# Patch the location of gfile
tf.gfile = tf.io.gfile

## Model preparation 

### Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing the path.

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

### Loader

In [ ]:
def load_custom_model(model_dir):
  model_dir = pathlib.Path(model_dir)/"saved_model"

  # Fix for "TypeError: load() missing 2 required positional arguments: 'tags' and 'export_dir'"
  # Reference: https://stackoverflow.com/questions/58683841/problem-with-running-object-detection-tutorial-typeerror-load-missing-2-requi
  # Reference: https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/saved_model/load_v2
  model = tf.saved_model.load_v2(str(model_dir))
  model = model.signatures['serving_default']

  return model


def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  # Print model checkpoint directory
  print(model_dir)
  !ls {model_dir}

  return load_custom_model(model_dir)

### Generate label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

Download dataset

In [ ]:
# Download dataset
!wget -nc https://data.broadinstitute.org/bbbc/BBBC041/malaria.zip
!unzip -u '{CWD}/malaria.zip' -x '__MACOSX/*'

Parse records and create label map

In [ ]:
import contextlib2
import json
import tensorflow as tf

from object_detection.dataset_tools import tf_record_creation_util
from object_detection.utils import dataset_util


def get_simplified_image_format(filename: str) -> str:
  extension = filename[-3:]

  if extension == 'jpg':
    return 'jpeg'
  else:
    return extension


def get_simplified_class(class_name: str) -> str:
  rbc_classes = {'red blood cell', 'trophozoite', 'schizont', 'ring', 'gametocyte', 'difficult'}

  if class_name in rbc_classes:
    return 'red blood cell'
  else:
    # Other (i.e. leukocyte)
    return class_name


def create_tf_example(example):
  # TODO(user): Populate the following variables from your example.
  height = example['height']  # Image height
  width = example['width']  # Image width
  # Filename of the image. Empty if image is not from file
  filename = example['filename']
  encoded_image_data = example['encoded_image_data']  # Encoded image bytes
  image_format = example['image_format']  # b'jpeg' or b'png'

  # List of normalized left x coordinates in bounding box (1 per box)
  xmins = example['xmins']
  # List of normalized right x coordinates in bounding box (1 per box)
  xmaxs = example['xmaxs']
  # List of normalized top y coordinates in bounding box (1 per box)
  ymins = example['ymins']
  # List of normalized bottom y coordinates in bounding box (1 per box)
  ymaxs = example['ymaxs']
  # List of string class name of bounding box (1 per box)
  classes_text = example['classes_text']
  # List of integer class id of bounding box (1 per box)
  classes = example['classes']

  tf_example = tf.train.Example(features=tf.train.Features(feature={
    'image/height': dataset_util.int64_feature(height),
    'image/width': dataset_util.int64_feature(width),
    'image/filename': dataset_util.bytes_feature(filename),
    'image/source_id': dataset_util.bytes_feature(filename),
    'image/encoded': dataset_util.bytes_feature(encoded_image_data),
    'image/format': dataset_util.bytes_feature(image_format),
    'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
    'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
    'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
    'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
    'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
    'image/object/class/label': dataset_util.int64_list_feature(classes),
  }))
  return tf_example


# TODO(user): Write code to read in your dataset to examples variable
TRAINING_JSON_PATH = f'{CWD}/malaria/training.json'
TEST_JSON_PATH = f'{CWD}/malaria/test.json'
DATASET_PATHS = [TRAINING_JSON_PATH, TEST_JSON_PATH]
NUM_SHARDS = 10
OUTPUT_PATHS = [f'{CWD}/malaria/training.record', f'{CWD}/malaria/test.record']
MODEL_DIR = f'{CWD}/rbc_model'
LABEL_MAP_PATH = f'{MODEL_DIR}/classes.pbtxt'
label_map = {}
next_class_index = 1
max_height = 0
max_width = 0
max_boxes = 0

# Parse records
for i in range(len(DATASET_PATHS)):
  output_path = OUTPUT_PATHS[i]

  with open(DATASET_PATHS[i]) as f:
    data = json.load(f)

  with contextlib2.ExitStack() as tf_record_close_stack:
    output_tfrecords = tf_record_creation_util.open_sharded_output_tfrecords(tf_record_close_stack, output_path, NUM_SHARDS)

    for j, entry in enumerate(data):
      # Add to label map
      # for obj in entry['objects']:
      #   if get_simplified_class(obj['category']) not in label_map:
      #     label_map[get_simplified_class(obj['category'])] = next_class_index
      #     next_class_index += 1

      label_map['red blood cell'] = 1 # Only rbc in label map

      # Encode image data
      with tf.gfile.GFile(f"{CWD}/malaria{entry['image']['pathname']}", 'rb') as fid:
        encoded_image_data = fid.read()

      # Only include bounding box of red blood cells
      dataset_format = {
        'height': entry['image']['shape']['r'],
        'width': entry['image']['shape']['c'],
        'filename': entry['image']['pathname'].encode(),
        'encoded_image_data': encoded_image_data,
        'image_format': get_simplified_image_format(entry['image']['pathname']).encode(),
        'xmins': [obj['bounding_box']['minimum']['c'] / entry['image']['shape']['c'] for obj in entry['objects'] if get_simplified_class(obj['category']) == 'red blood cell'],
        'xmaxs': [obj['bounding_box']['maximum']['c'] / entry['image']['shape']['c'] for obj in entry['objects'] if get_simplified_class(obj['category']) == 'red blood cell'],
        'ymins': [obj['bounding_box']['minimum']['r'] / entry['image']['shape']['r'] for obj in entry['objects'] if get_simplified_class(obj['category']) == 'red blood cell'],
        'ymaxs': [obj['bounding_box']['maximum']['r'] / entry['image']['shape']['r'] for obj in entry['objects'] if get_simplified_class(obj['category']) == 'red blood cell'],
        'classes_text': [get_simplified_class(obj['category']).encode() for obj in entry['objects'] if get_simplified_class(obj['category']) == 'red blood cell'],
        'classes': [label_map[get_simplified_class(obj['category'])] for obj in entry['objects'] if get_simplified_class(obj['category']) == 'red blood cell']
      }

      # Update config variables
      max_height = max(dataset_format['height'], max_height)
      max_width = max(dataset_format['width'], max_width)
      max_boxes = max(len(entry['objects']), max_boxes)

      # Add to shards
      tf_example = create_tf_example(dataset_format)
      output_shard_index = j % NUM_SHARDS

      output_tfrecords[output_shard_index].write(tf_example.SerializeToString())

# Create label map
!mkdir -p {MODEL_DIR}

with open(LABEL_MAP_PATH, 'w') as f:
  first_label = True

  for label in label_map:
    f.write('item {\n') if first_label else f.write('\nitem {\n')
    f.write(f'  id: {label_map[label]}\n')
    f.write(f'  name: "{label}"\n')
    f.write('}\n')

    first_label = False

print(f'(max_height, max_width) = ({max_height}, {max_width})')
print('Verify "ssd.config" matches the following:')
print(f'train_/eval_input_reader => label_map_path: "{LABEL_MAP_PATH}"')
print(f'train_input_reader => input_path: "{OUTPUT_PATHS[0]}-?????-of-00010"')
print(f'train_input_reader => max_number_of_boxes: {max_boxes}')
print(f'eval_input_reader => input_path: "{OUTPUT_PATHS[1]}-?????-of-00010"')

In [ ]:
# List of the strings that is used to add correct label for each box.
category_index = label_map_util.create_category_index_from_labelmap(LABEL_MAP_PATH, use_display_name=True)

print(f'Save label map: "{LABEL_MAP_PATH}"')
print()
category_index

### Save these files
- Label map

## Train custom model

### Required files
- Images
- Pipeline configuration file

### Load base model

In [ ]:
model_name = 'ssd_mobilenet_v2_coco_2018_03_29'
detection_model = load_model(model_name)

# ssd.config file => fine_tune_checkpoint: "/root/.keras/datasets/ssd_mobilenet_v2_coco_2018_03_29/model.ckpt"

### Load custom config file

In [ ]:
PIPELINE_CONFIG_PATH = f'{CWD}/ssd.config'
MODEL_DIR = f'{CWD}/training'
NUM_STEPS = 80000

### Run training script

In [ ]:
!python {CWD}/models/research/object_detection/model_main.py \
    --pipeline_config_path={PIPELINE_CONFIG_PATH} \
    --model_dir={MODEL_DIR} \
    --alsologtostderr \
    --num_train_steps={NUM_STEPS} \
    2>&1 | tee {MODEL_DIR}/output.log

### Export trained model

List available checkpoints

In [ ]:
!ls {CWD}/training | grep model.ckpt

In [ ]:
PIPELINE_CONFIG_PATH = f'{CWD}/ssd.config'
TRAINED_CKPT_PREFIX = f'{CWD}/training/model.ckpt-###' # Set "###" to latest checkpoint file

Export Model

In [ ]:
# Reference: https://github.com/tensorflow/models/blob/master/research/object_detection/export_inference_graph.py
INPUT_TYPE='image_tensor'
MODEL_EXPORT_DIR = f'{CWD}/rbc_model'

!python {CWD}/models/research/object_detection/export_inference_graph.py \
    --input_type={INPUT_TYPE} \
    --pipeline_config_path={PIPELINE_CONFIG_PATH} \
    --trained_checkpoint_prefix={TRAINED_CKPT_PREFIX} \
    --output_directory={MODEL_EXPORT_DIR}

In [ ]:
!ls {MODEL_EXPORT_DIR}

Export to TensorFlow Lite

In [ ]:
# Reference: https://stackoverflow.com/questions/54823475/tensorflow-object-detection-convert-pb-file-to-tflite
# Reference: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/running_on_mobile_tensorflowlite.md
# Reference: https://github.com/tensorflow/models/blob/master/research/object_detection/export_tflite_ssd_graph.py
# Reference: https://stackoverflow.com/questions/58052869/tf-lite-object-detection-only-returning-10-detections
MOBILE_MODEL_EXPORT_DIR = f'{CWD}/rbc_mobile_model'
MAX_DETECTIONS = 300

!python {CWD}/models/research/object_detection/export_tflite_ssd_graph.py \
  --pipeline_config_path={PIPELINE_CONFIG_PATH} \
  --trained_checkpoint_prefix={TRAINED_CKPT_PREFIX} \
  --output_directory={MOBILE_MODEL_EXPORT_DIR} \
  --max_detections={MAX_DETECTIONS} \
  --add_postprocessing_op=true

In [ ]:
!ls {MOBILE_MODEL_EXPORT_DIR}

Convert to .tflite

In [ ]:
TFLITE_OUTPUT_PATH = f'{CWD}/rbc_mobile_model.tflite'
GRAPH_DEF_FILE = f'{MOBILE_MODEL_EXPORT_DIR}/tflite_graph.pb'
INPUT_ARRAYS = 'normalized_input_image_tensor'
OUTPUT_ARRAYS = "'TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3'"
INPUT_SHAPES = '1,300,300,3'

In [ ]:
# Reference: https://stackoverflow.com/questions/50632152/tensorflow-convert-pb-file-to-tflite-using-python
!tflite_convert \
  --output_file={TFLITE_OUTPUT_PATH} \
  --graph_def_file={GRAPH_DEF_FILE} \
  --input_arrays={INPUT_ARRAYS} \
  --output_arrays={OUTPUT_ARRAYS} \
  --input_shape={INPUT_SHAPES} \
  --allow_custom_ops

In [ ]:
!ls | grep *.tflite

### Save these files
- Training folder
- Model export folder
- tflite model export folder
- `.tflite` file

Compress training folder

In [ ]:
!tar -cvzf {CWD}/training.tar.gz {MODEL_DIR}
print()
print('Save me!')

Compress model export folder

In [ ]:
!tar -cvzf {CWD}/rbc_model.tar.gz {MODEL_EXPORT_DIR}
print()
print('Save me!')

Compress mobile model export folder

In [ ]:
!tar -cvzf {CWD}/rbc_mobile_model.tar.gz {MOBILE_MODEL_EXPORT_DIR}
print()
print('Save me!')

# Detection

## Required Files
- Images
- Label map
- Exported model

## Setup

Upgrade to tf v2

In [ ]:
# Fix for "TypeError: int() argument must be a string, a bytes-like object or a number, not 'Tensor'" during training
!pip install -U tensorflow=="2.*"
!pip install -U tensorflow-gpu=="2.*"

Re-Imports

In [ ]:
import os
import pathlib


CWD = pathlib.Path.cwd()

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

print(f'Current Working Directory: "{CWD}"')

In [ ]:
# Increase default font size of bounding box
!cp {CWD}/visualization_utils_fix.py {CWD}/models/research/object_detection/utils/visualization_utils.py

Compile protobufs and install the object_detection package

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [ ]:
%%bash
cd models/research
pip install .

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import json

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

Import the object detection module.

In [ ]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

Patches:

In [ ]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

Download dataset

In [ ]:
# Download dataset
!wget -nc https://data.broadinstitute.org/bbbc/BBBC041/malaria.zip
!unzip -u '{CWD}/malaria.zip' -x '__MACOSX/*'

## Load custom model

In [ ]:
def load_custom_model(model_dir):
  model_dir = pathlib.Path(model_dir)/"saved_model"

  # Fix for "TypeError: load() missing 2 required positional arguments: 'tags' and 'export_dir'"
  # Reference: https://stackoverflow.com/questions/58683841/problem-with-running-object-detection-tutorial-typeerror-load-missing-2-requi
  # Reference: https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/saved_model/load_v2
  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model


def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  # Print model checkpoint directory
  print(model_dir)
  !ls {model_dir}

  return load_custom_model(model_dir)

Extract model if necessary (uncomment to run)

In [ ]:
# MODEL_NAME = 'rbc_model'

# !mkdir -p {CWD}/{MODEL_NAME}
# !tar -xvf {CWD}/{MODEL_NAME}.tar.gz -C {CWD}/{MODEL_NAME}

Load model

In [ ]:
# model_name = 'ssd_mobilenet_v2_coco_2018_03_29'
# detection_model = load_model(model_name)
MODEL_EXPORT_DIR = f'{CWD}/rbc_model'
detection_model = load_custom_model(MODEL_EXPORT_DIR)

Check the model's input signature, it expects a batch of 3-color images of type uint8: 

In [ ]:
# Verify: "shape=(None, None, None, 3) dtype=uint8>"
print(detection_model.inputs)

And retuns several outputs:

In [ ]:
detection_model.output_dtypes

In [ ]:
detection_model.output_shapes

## Helper functions

Add a wrapper function to call the model, and cleanup the outputs:

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)
  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

Run it on each test image and show the results:

In [ ]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  # print(output_dict)
  display(Image.fromarray(image_np))

## Evaluate model

Set test images paths

In [ ]:
DATASET_PATH = f'{CWD}/malaria'
TRAINING_JSON_PATH = f'{DATASET_PATH}/training.json'
TEST_JSON_PATH = f'{DATASET_PATH}/test.json'

# Load training images
with open(TRAINING_JSON_PATH) as f:
  data = json.load(f)

TRAINING_IMAGES_PATHS = [pathlib.Path(f"{DATASET_PATH}{entry['image']['pathname']}") for entry in data]

# Load test images
with open(TEST_JSON_PATH) as f:
  data = json.load(f)

TEST_IMAGES_PATHS = [pathlib.Path(f"{DATASET_PATH}{entry['image']['pathname']}") for entry in data]

print(TRAINING_IMAGES_PATHS)
print(f'Training images count: {len(TRAINING_IMAGES_PATHS)}')
print(TEST_IMAGES_PATHS)
print(f'Test images count: {len(TEST_IMAGES_PATHS)}')

Load label map

In [ ]:
LABEL_MAP_PATH = f'{MODEL_EXPORT_DIR}/classes.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(LABEL_MAP_PATH, use_display_name=True)

Run random test image through model

In [ ]:
import random

# random_image_path = random.choice(TRAINING_IMAGES_PATHS)
random_image_path = random.choice(TEST_IMAGES_PATHS)

show_inference(detection_model, random_image_path)